In [ ]:
import sagemaker
import os
from datetime import datetime
from sagemaker.utils import sagemaker_timestamp
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

### SETUP

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
ROLE = get_execution_role()
SESSION = sagemaker.Session()
TIMESTAMP = datetime.now().strftime("%d-%m-%Y----%H-%M-%S")

### TRAINING VARIABLES

In [ ]:
INSTANCE_TYPE = 'ml.p3.8xlarge'
INSTANCE_COUNT = 1
TRAINING_SCRIPT = 'train.py'
JOB_ID = 'training' 


BUCKET = 'csu-team-04-training'
INFER_BUCKET = 'csu-team04-inference-model'

### HYPERPARAMETERS

In [ ]:
job_name = f'{JOB_ID}----{TIMESTAMP}'

hyperparameters = {
    "epochs": 5,
    "train-bucket": BUCKET,
    "infer-bucket": INFER_BUCKET,
    "job_name": job_name,
}

### TRAIN

In [ ]:
estimator = TensorFlow(
    role=ROLE,
    entry_point=TRAINING_SCRIPT,
    output_path=f's3://{BUCKET}/training_output',
    instance_count=INSTANCE_COUNT,
    instance_type=INSTANCE_TYPE,
    hyperparameters=hyperparameters,
    py_version='py37',
    framework_version = '2.3',
    source_dir='src'
)

estimator.fit({"train": f's3://{BUCKET}'}, job_name=job_name)